# Bray–Curtis dissimilarity between regions defined by important historical boundaries

Compare historical regions in terms of their building composition using Bray–Curtis dissimilarity statistic.

In [1]:
import geopandas as gpd
import pandas as pd
import scipy.stats as stats
import shapely
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from libpysal import graph
from scipy.spatial import distance
import os

import preprocess_data

Retrieve building dataset and levels of clusters

In [ ]:
buildings, levels = preprocess_data.get_buildings("/data/uscuni-ulce/boundaries_of_change/classification/")

List layers of influential boundaries across Central European history

In [ ]:
gpd.list_layers("/data/uscuni-ulce/boundaries_of_change/impact_boundaries.gpkg")

In [ ]:
boundary_list = list(gpd.list_layers("/data/uscuni-ulce/boundaries_of_change/impact_boundaries.gpkg").name)
boundary_list.remove("1812_political") #to be fixed
boundary_list.remove("1900_germans") #to be fixed
boundary_list.remove("1300_ostsiedlung") #to be fixed

Define area of interest (Central Europe sans Hungary)

In [ ]:
aoi = gpd.read_file("/data/uscuni-ulce/boundaries_of_change/impact_boundaries.gpkg", layer="1994_political")
aoi = aoi.dissolve()
aoi.plot()

**Example**: Consider `1994_political` map

In [ ]:
layer_of_interest = boundary_list[-1]

Retrieve building counts per region and normalize

In [ ]:
building_counts = preprocess_data.get_building_counts_per_region(buildings, gpd.read_file("/data/uscuni-ulce/boundaries_of_change/impact_boundaries.gpkg", layer=layer_of_interest))

In [ ]:
normalized_building_counts = preprocess_data.normalize_building_counts(building_counts)

In [ ]:
normalized_building_counts

Compute pairwise Bray–Curtis dissimilarity matrix

In [ ]:
def compute_bc_matrix(count_table):
    bray_curtis_dist = distance.pdist(count_table, metric='braycurtis')
    bray_curtis_matrix = distance.squareform(bray_curtis_dist)
    bray_curtis_df = pd.DataFrame(bray_curtis_matrix, index=count_table.T.columns, columns=count_table.T.columns)
    return bray_curtis_df

In [ ]:
bray_curtis_matrix = compute_bc_matrix(normalized_building_counts)
bray_curtis_matrix

Plot as a heatmap

In [ ]:
sns.heatmap(bray_curtis_matrix, annot=True)

Compute Bray–Curtis matrix for each cluster level

In [ ]:
def generate_bc_heatmaps_for_boundary(level_list, boundary_name):

    fig, axs = plt.subplots(2, 3, figsize=(18, 10))
    axs = axs.flatten()
    
    for level, ax in zip(level_list, axs):
        bc = pd.read_parquet(f'/data/uscuni-ulce/boundaries_of_change/bc_matrices/bc_{boundary_name}_{level}.pq')
        sns.heatmap(bc, annot=True, ax=ax)
        ax.set_title(level)

    fig.suptitle(boundary_name)
    fig.tight_layout()

In [ ]:
generate_bc_heatmaps_for_boundary(levels, layer_of_interest)

## Run to generate Bray–Curtis matrices for each level and historical layer, save to parquet

In [13]:
from tqdm.auto import tqdm

def generate_bc_matrices(list_of_boundaries, level_list):
    bc_matrices_path = "/data/uscuni-ulce/boundaries_of_change/bc_matrices/"
    if not os.path.exists(bc_matrices_path):
        os.mkdir(bc_matrices_path)
    for bnd in tqdm(list_of_boundaries, desc=f"Processing boundaries..."):
        boundary = gpd.read_file("/data/uscuni-ulce/boundaries_of_change/impact_boundaries.gpkg", layer=bnd)
        
        for level in tqdm(level_list, desc=f"Generating matrices for {bnd}"):
            building_counts = preprocess_data.get_building_counts_per_region(buildings, boundary, level)
            building_counts_norm = preprocess_data.normalize_building_counts(building_counts)
            
            bray_curtis_matrix = compute_bc_matrix(building_counts_norm)
            output_path = os.path.join(bc_matrices_path, f'bc_{bnd}_{level}.pq')
            bray_curtis_matrix.to_parquet(output_path)

In [14]:
generate_bc_matrices(boundary_list, levels)

Processing boundaries...: 100%|████████████████████████████████████████████████████████| 15/15 [48:26<00:00, 193.78s/it]
